In [ ]:
from llama_sim import LlamaForCausalLM
#from transformers import LlamaForCausalLM
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5"
hf_llm = LlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-70B-Instruct", device_map="auto", torch_dtype=torch.bfloat16)



In [ ]:
import numpy as np
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)
hf_llm = hf_llm.eval()
hf_llm.config.K = 12
hf_llm.config.L = 150
hf_llm.config.cache_mode = "anns_es"
hf_llm.config.window = 16
hf_llm.set_sparse_attn(sparse=0.02, random_sparse=0.2, window_size=16, kernel_size=5)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-70B-Instruct", use_fast=False)

In [ ]:
input_text = """
System: You are an assistant. Your goal is to be helpful, friendly and answer the questions as more as you can. 
User: Hello, I want to test your mathematic ability, nice to meet you! I have a problem for you.
@<<<assign v0 = v2 - 1>>>@@<<<assign v1 = v3 + 1>>>@@<<<assign v2 = 1>>>@@<<<assign v3 = v2>>>@
**Question:**
Can you tell me which variables are equal to 2 in those assignments in '<<<>>>'? The values of those variables can be used before definition as they are constant.
You can solve the problem in a step-by-step way.
Assistant: 
"""
input_tokens = tokenizer.encode(input_text, return_tensors="pt")
print(input_tokens, len(input_tokens[0]))

In [ ]:
output = hf_llm.generate(inputs=input_tokens, do_sample=False, max_new_tokens=1024)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)